In [1]:
import numpy as np
import pandas as pd

from os import scandir
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm

In [2]:
rutaASI='C:\\Users\\boliv\\Desktop\\Ejercicio_Python\\Codigo_MCLJ_PCA\\Datos_SAC_LJMC\\ASI'
rutaCO= "C:\\Users\\boliv\\Desktop\\Ejercicio_Python\\Codigo_MCLJ_PCA\\Datos_SAC_LJMC\\CO"
# función que regresa todos los archivos en la ruta path
def dimeArchivos(path): 
    return [obj.name for obj in scandir(path) if obj.is_file()]

#Listas con los nombres de los archivos en la ruta dada
ListaArchASI = dimeArchivos(rutaASI)
ListaArchCO  = dimeArchivos(rutaCO)

#print(len(ListaArchASI))
#print(ListaArchASI)

#Crea lista con las primeras dos letras de cada archivo, correspodientes a los nombres de los sujetos
Name_suj_ASI = [x[0:2] for x in ListaArchASI]
Name_suj_CO  = [x[0:2] for x in ListaArchCO]

#print(Name_suj)

#Crea dict para eliminar duplicados y los convierte en lista
Name_suj_ASI = list(dict.fromkeys(Name_suj_ASI)) # ['BG','CR','EN','GF','GV','IF','IM','JE','JU','JV','LM','MI','RB','RR','VC','VV']
Name_suj_CO = list(dict.fromkeys(Name_suj_CO))   # ['AG','AS','AZ','BE','CV','EG','GM','GV','LO','LP','LR','MN','MS','PA','TR']

#print(Name_suj_ASI,type(Name_suj_ASI))
#print(Name_suj_CO, type(Name_suj_CO))

#Selecciona aleatoriamente 12 sujetos para crear los conjuntos de prueba
CO_15 = sorted(Name_suj_CO)
ASI_15 = random.sample(Name_suj_ASI,15)
ASI_out = [x for x in Name_suj_ASI if x not in ASI_15]

ASI_15=sorted(ASI_15)
ASI_out= sorted(ASI_out)

In [3]:
print(ASI_15)
print(CO_15)
print(ASI_out)

['BG', 'CR', 'EN', 'GF', 'GV', 'IF', 'IM', 'JE', 'JU', 'JV', 'LM', 'MI', 'RB', 'VC', 'VV']
['AG', 'AS', 'AZ', 'BE', 'CV', 'EG', 'GM', 'GV', 'LO', 'LP', 'LR', 'MN', 'MS', 'PA', 'TR']
['RR']


In [4]:
#Lista de las rutas de los archivos por conjunto (train/test)
Rutas_ASI_15 = [rutaASI+"\\"+x for x in ListaArchASI if x[0:2] in     ASI_15]
Rutas_ASI_out  = [rutaASI+"\\"+x for x in ListaArchASI if x[0:2] not in ASI_15]
Rutas_CO_15  = [rutaCO+"\\"+x  for x in ListaArchCO  if x[0:2] in     CO_15]

#for i in range (len(Rutas_ASI_out)):
#    print(Rutas_ASI_out[i])

In [5]:
# Arma arreglo de natriz SAC
Arreglo_ASI=[]
Arreglo_CO=[]

asi_clase = np.array ([-1])
co_clase = np.array ([1])


for i in Rutas_ASI_15: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, asi_clase))
    Arreglo_ASI = np.concatenate((Arreglo_ASI,array_sac))

Arreglo_ASI = Arreglo_ASI.reshape(len(ASI_15)*14,92)

for i in Rutas_CO_15: # ruta de archivo en conjunto dado
    matriz_i = np.loadtxt(open(i))
    array_sac=[]
    for j in range(len(matriz_i)):
        array_sac = np.concatenate((array_sac, matriz_i[j,j+1:]))
    array_sac = np.concatenate((array_sac, co_clase))
    Arreglo_CO = np.concatenate((Arreglo_CO,array_sac))

Arreglo_CO = Arreglo_CO.reshape(len(CO_15)*14,92)

#print(Arreglo_ASI[209])
        
Array_SVM = np.concatenate((Arreglo_ASI,Arreglo_CO), axis=0)
print(Array_SVM.shape)


(420, 92)


In [6]:
#Separación conjuntos entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(Array_SVM[:,0:91], Array_SVM[:,-1], test_size = 0.2, random_state=0)

In [7]:
print('Train_Set: ',X_train.shape, y_train.shape)

print('Test_Set: ',X_test.shape, y_test.shape)

Train_Set:  (336, 91) (336,)
Test_Set:  (84, 91) (84,)


In [8]:
#Intervalo de parámetros
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}

In [9]:
clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1)

In [10]:
# Se buscan los mejores parámetros para los conjuntos de entrenamiento
clf_grid.fit(X_train, y_train)

print("Best Parameters:\n", clf_grid.best_params_)
print("Best Estimators:\n", clf_grid.best_estimator_)

C:\Users\boliv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters:
 {'C': 10, 'gamma': 0.1}
Best Estimators:
 SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:    0.6s finished
C:\Users\boliv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [11]:
#Se ajusta el clasificador con los mejores parámetros encontrados
clf = svm.SVC(kernel='rbf', C=10, gamma=0.1)
clf.fit(X_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [12]:
# Se realizan las predicciones en conjuntos de prueba
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))

Accuracy: 98.80952380952381%


#### Módulo CrossValidation

In [16]:
from sklearn.model_selection import cross_val_score
clf2 = svm.SVC(kernel='rbf', gamma='auto')

# Separa los datos, ajusta el modelo, y calcula la precisión "cv" veces
scores2 = cross_val_score(clf2, Array_SVM[:,0:91], Array_SVM[:,-1], cv=10)
print(scores2)

#Cálculo de la media de las predicciones obtenidas
print("Accuracy: {0:.2F}%".format((scores2.mean())*100),"(+/- {0:0.2f}%)".format(scores2.std() * 2))

[0.69047619 0.54761905 0.76190476 0.61904762 0.73809524 0.71428571
 0.45238095 0.5        0.92857143 0.42857143]
Accuracy: 63.81% (+/- 0.30%)


In [ ]:
#[0.88095238 0.78571429 0.97619048 0.71428571 0.95238095 0.71428571 0.54761905 0.64285714 0.9047619  0.5952381 ]